# Filtrado, preparación e integración de datos

Esta notebook implementa un pipeline reproducible con **pandas** para:
- Renombrar columnas a `snake_case`.
- Convertir fechas a `datetime`.
- Filtrar viajes de **julio, agosto y septiembre de 2024**.
- Guardar un resultado intermedio en `data/interim/recorridos_3meses.csv`.
- Verificar la **clave de unión** (`id_usuario` o equivalente) en ambos datasets.
- Realizar el **join** entre `recorridos` filtrados y `usuarios`.
- Crear la columna **`duracion_viaje_minutos`**.
- Guardar el resultado final en `data/processed/recorridos_usuarios_3meses.csv`.

> 💡 *Sigue la estructura: textos explicativos breves antes de cada bloque de código y comentarios puntuales dentro del código.*


## 💡 Recomendaciones para el uso de notebooks

- Usar celdas de texto (Markdown) para documentar cada parte del proceso.
- Utilizar títulos y subtítulos (`#`, `##`, `###`) para organizar el contenido.
- Explicar brevemente **antes** de cada bloque de código qué se va a hacer y por qué.
- Incluir comentarios en el código solo para **aclaraciones puntuales**, no para repetir lo que ya está en el texto.
- Mantener el notebook **limpio y ejecutable de principio a fin**, sin errores.


## 1️⃣ Configuración

Define aquí las rutas de entrada/salida y los nombres de columnas clave.  
Ajusta los nombres si tus archivos usan otras etiquetas.


In [13]:
# --- Configuración del proyecto ---
from pathlib import Path

# Directorio raíz del proyecto
DIRECTORIO = '/Users/devlaptop//Documents/GitHub/machine-learning-unlu/tp00-retomando_impulso/'

# Rutas relativas al raíz del proyecto (ajusta si es necesario)
DATA_RAW = Path(DIRECTORIO + 'data/raw')
DATA_INTERIM = Path(DIRECTORIO + 'data/interim')
DATA_PROCESSED = Path(DIRECTORIO + 'data/processed')

# Nombres de archivos de entrada (ajusta si difieren)
F_USUARIOS = DATA_RAW / 'usuarios_ecobici_2024.csv'
F_RECORRIDOS = DATA_RAW / 'badata_ecobici_recorridos_realizados_2024.csv'

# Salidas
F_RECORRIDOS_FILTRADOS = DATA_INTERIM / 'recorridos_3meses.csv'
F_FINAL = DATA_PROCESSED / 'recorridos_usuarios_3meses.csv'

# Columnas clave (ajusta si difiere)
COL_ID_USUARIO = 'id_usuario'               # clave para el join
COL_DT_INICIO = 'fecha_origen_recorrido'    # datetime de inicio del viaje
COL_DT_FIN = 'fecha_destino_recorrido'      # datetime de fin del viaje

# Crear carpetas de salida si no existen
for p in [DATA_INTERIM, DATA_PROCESSED]:
    p.mkdir(parents=True, exist_ok=True)

print('Entradas:')
print('  -', F_USUARIOS)
print('  -', F_RECORRIDOS)
print('\nSalidas:')
print('  -', F_RECORRIDOS_FILTRADOS)
print('  -', F_FINAL)

Entradas:
  - /Users/devlaptop/Documents/GitHub/machine-learning-unlu/tp00-retomando_impulso/data/raw/usuarios_ecobici_2024.csv
  - /Users/devlaptop/Documents/GitHub/machine-learning-unlu/tp00-retomando_impulso/data/raw/badata_ecobici_recorridos_realizados_2024.csv

Salidas:
  - /Users/devlaptop/Documents/GitHub/machine-learning-unlu/tp00-retomando_impulso/data/interim/recorridos_3meses.csv
  - /Users/devlaptop/Documents/GitHub/machine-learning-unlu/tp00-retomando_impulso/data/processed/recorridos_usuarios_3meses.csv


## 2️⃣ Carga de datos

Leemos los dos CSV desde `data/raw/`. Si tus archivos están en otro formato o nombre, ajusta la configuración previa.


In [10]:
import pandas as pd

usuarios = pd.read_csv(F_USUARIOS)
recorridos = pd.read_csv(F_RECORRIDOS)

print('usuarios:', usuarios.shape)
print('recorridos:', recorridos.shape)

usuarios: (197077, 5)
recorridos: (3559284, 17)


## 3️⃣ Estandarización de nombres a `snake_case`

Unificamos el estilo de columnas para evitar errores en merges o referencias posteriores.


In [11]:
import re

def to_snake(s: str) -> str:
    s = s.strip()
    s = re.sub(r'[\s\-\/]+', '_', s)            # espacios y separadores a guión bajo
    s = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', s)  # camelCase/PascalCase a snake
    s = re.sub(r'__+', '_', s)                  # colapsar dobles guiones bajos
    s = re.sub(r'[^a-zA-Z0-9_]', '', s)         # quitar caracteres no válidos
    return s.lower()

def snakecase_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [to_snake(c) for c in df.columns]
    return df

usuarios = snakecase_columns(usuarios)
recorridos = snakecase_columns(recorridos)

print('Columnas usuarios:', usuarios.columns.tolist())
print('Columnas recorridos:', recorridos.columns.tolist())

Columnas usuarios: ['id_usuario', 'genero_usuario', 'edad_usuario', 'fecha_alta', 'hora_alta']
Columnas recorridos: ['id_recorrido', 'duracion_recorrido', 'fecha_origen_recorrido', 'id_estacion_origen', 'nombre_estacion_origen', 'direccion_estacion_origen', 'long_estacion_origen', 'lat_estacion_origen', 'fecha_destino_recorrido', 'id_estacion_destino', 'nombre_estacion_destino', 'direccion_estacion_destino', 'long_estacion_destino', 'lat_estacion_destino', 'id_usuario', 'modelo_bicicleta', 'genero']


## 4️⃣ Conversión de fechas a `datetime`

Convertimos las columnas de fecha/hora definidas en la configuración.  
Si tus nombres reales difieren, actualízalos arriba.


In [14]:
for col in [COL_DT_INICIO, COL_DT_FIN]:
    if col in recorridos.columns:
        recorridos[col] = pd.to_datetime(recorridos[col], errors='coerce', utc=False)
    else:
        raise KeyError(f'No se encontró la columna de fecha/hora: {col}')

# Diagnóstico rápido
print(recorridos[[COL_DT_INICIO, COL_DT_FIN]].dtypes)

fecha_origen_recorrido     datetime64[ns]
fecha_destino_recorrido    datetime64[ns]
dtype: object


## 5️⃣ Filtro de viajes (julio–septiembre 2024)

Nos quedamos con recorridos cuyo **inicio** esté entre el **1 de julio** y el **30 de septiembre de 2024**.
Luego persistimos el resultado intermedio en `data/interim/recorridos_3meses.csv`.


In [15]:
inicio = pd.Timestamp('2024-07-01 00:00:00')
fin = pd.Timestamp('2024-09-30 23:59:59')

mask = (recorridos[COL_DT_INICIO] >= inicio) & (recorridos[COL_DT_INICIO] <= fin)
recorridos_filtrados = recorridos.loc[mask].copy()

print('Filtrados:', recorridos_filtrados.shape)
recorridos_filtrados.to_csv(F_RECORRIDOS_FILTRADOS, index=False)
print('Guardado intermedio ->', F_RECORRIDOS_FILTRADOS)

Filtrados: (831984, 17)
Guardado intermedio -> /Users/devlaptop/Documents/GitHub/machine-learning-unlu/tp00-retomando_impulso/data/interim/recorridos_3meses.csv


## 6️⃣ Verificación de la clave de unión

Comprobamos que la clave (`id_usuario` o equivalente) exista en ambos datasets y revisamos:
- **Valores nulos** en la clave.
- **Duplicados** (unicidad) en `usuarios` para la clave.
- **Coverage**: cuántos `id_usuario` de `recorridos_filtrados` están presentes en `usuarios`.


In [16]:
# Existencia de la clave
for df_name, df in [('usuarios', usuarios), ('recorridos_filtrados', recorridos_filtrados)]:
    if COL_ID_USUARIO not in df.columns:
        raise KeyError(f'No se encontró {COL_ID_USUARIO} en {df_name}')

# Nulos
print('Nulos en clave:')
print('  usuarios:', usuarios[COL_ID_USUARIO].isna().sum())
print('  recorridos_filtrados:', recorridos_filtrados[COL_ID_USUARIO].isna().sum())

# Duplicados en la tabla de dimensión (usuarios)
dups = usuarios[COL_ID_USUARIO].duplicated().sum()
print('Duplicados de clave en usuarios:', dups)

# Cobertura
ids_rec = set(recorridos_filtrados[COL_ID_USUARIO].dropna().unique())
ids_usu = set(usuarios[COL_ID_USUARIO].dropna().unique())
coverage = len(ids_rec & ids_usu) / max(1, len(ids_rec))
print(f'Coverage de id_usuario (recorridos en usuarios): {coverage:.2%}  ({len(ids_rec & ids_usu)}/{len(ids_rec)})')

Nulos en clave:
  usuarios: 0
  recorridos_filtrados: 0
Duplicados de clave en usuarios: 0
Coverage de id_usuario (recorridos en usuarios): 45.05%  (47703/105884)


## 7️⃣ Integración (join)

Hacemos un **left join** de `recorridos_filtrados` con `usuarios` por la clave.  
Así mantenemos todos los recorridos y anexamos atributos del usuario.


In [25]:
cols_usuarios = [c for c in usuarios.columns if c != COL_ID_USUARIO]
df_integrado = recorridos_filtrados.merge(usuarios, on=COL_ID_USUARIO, how='left', validate='m:1')

print('Integrado:', df_integrado.shape)
print('Columnas añadidas de usuarios:', [c for c in cols_usuarios if c in df_integrado.columns])

Integrado: (831984, 21)
Columnas añadidas de usuarios: ['genero_usuario', 'edad_usuario', 'fecha_alta', 'hora_alta']


## 8️⃣ Columna derivada `duracion_viaje_minutos`

Calculamos la duración en minutos como la diferencia entre `fecha_fin` y `fecha_inicio`.  
Si falta alguno de los extremos o la resta no es válida, el resultado será `NaN`.


In [26]:
import numpy as np

if COL_DT_INICIO not in df_integrado.columns or COL_DT_FIN not in df_integrado.columns:
    raise KeyError('No se encuentran las columnas de fecha de inicio/fin en el DF integrado.')

dur = (df_integrado[COL_DT_FIN] - df_integrado[COL_DT_INICIO]).dt.total_seconds() / 60.0
df_integrado['duracion_viaje_minutos'] = dur.replace([np.inf, -np.inf], np.nan)

print('Descripción de duracion_viaje_minutos:')
print(df_integrado['duracion_viaje_minutos'].describe())

Descripción de duracion_viaje_minutos:
count    831984.000000
mean         21.408908
std         190.600092
min           0.000000
25%           7.850000
50%          14.133333
75%          23.766667
max       42852.750000
Name: duracion_viaje_minutos, dtype: float64


## 9️⃣ Guardado del resultado final

Persistimos el dataset enriquecido en `data/processed/recorridos_usuarios_3meses.csv`.


In [23]:
df_integrado.to_csv(F_FINAL, index=False)
print('Guardado final ->', F_FINAL)

Guardado final -> /Users/devlaptop/Documents/GitHub/machine-learning-unlu/tp00-retomando_impulso/data/processed/recorridos_usuarios_3meses.csv


## 🔟 Resumen y chequeos finales

Mostramos un resumen rápido para verificar el pipeline.


In [27]:
print('Filas finales:', df_integrado.shape[0])
print('Columnas finales:', df_integrado.shape[1])

# Nulos por columna (top 10)
nulos = df_integrado.isnull().sum().sort_values(ascending=False).head(10)
print('\nTop 10 columnas con más nulos:')
print(nulos)

print('\nTipos de datos (primeras 20 columnas):')
print(df_integrado.dtypes.head(20))

Filas finales: 831984
Columnas finales: 22

Top 10 columnas con más nulos:
hora_alta               540976
fecha_alta              540976
edad_usuario            540976
genero_usuario          540976
genero                    2907
id_recorrido                 0
duracion_recorrido           0
modelo_bicicleta             0
id_usuario                   0
lat_estacion_destino         0
dtype: int64

Tipos de datos (primeras 20 columnas):
id_recorrido                           int64
duracion_recorrido                     int64
fecha_origen_recorrido        datetime64[ns]
id_estacion_origen                     int64
nombre_estacion_origen                object
direccion_estacion_origen             object
long_estacion_origen                 float64
lat_estacion_origen                  float64
fecha_destino_recorrido       datetime64[ns]
id_estacion_destino                    int64
nombre_estacion_destino               object
direccion_estacion_destino            object
long_estacion_destino 